In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt 
import sklearn
import torch

# Markov Decision Process

Typically one has the following flow chart
$$S_t \rightarrow A_t \rightarrow R_{t+1}, S_{t+1}$$
where $S_t$ is the state process and $A_t$ is the action and $R_{t+1}$ is the reward. We point out that sometimes one may only have partial observation of the state process $S_t$, in this case the filtering method can be applied to attack such problem. However, for simplicity and to focus on the main idea here, we assume one can observe the entire state. 

What is interesting sometimes is that when the agent acts, the large rewards or actions can impact the environment. For example in the financial market, the major player can usually make a large impact on the market. 

A policy is a rule to make actions
$$\pi = \mathbf{P}(a|s)$$
A policy can be deterministic, in this case, it is simply a function of the state $s$. 
$$\pi = f(s), \  \text{or} \ \ \pi = \sum^L_{i} a_i \delta(s_i)$$

`Should insert some pics here`

The reward received and next state of the agent would then be a probability distribution over the possible values or reward  and next states
$$\mathbb{P}(s', r)=\mathbb{P}(S_t=s', R_t=r| S_{t-1}=s, A_{t-1}=a)$$
by tracing over the parameter $r$, we obtain the following probability distribution over the next state $s'$
$$\mathbb{P}(s'| S_{t-1}=s, A_{t-1}=a)=\sum_{r}\mathbb{P}(S_t=s', R_t=r| S_{t-1}=s, A_{t-1}=a)$$



## Bellman's equation

We define $G_t$ to be the net lookahead reward:
$$G_t=R_{t+1} +rR_{t+2}+r^2R_{t+3}+...+r^{T-t-1}R_{T}$$
Then, clearly this function depends on the current state $S_t$ and the future trajectory (policy).
Define the value function,
$$
\begin{equation}
v_{\pi}(s)=\mathbb{E}_{\pi}[G_t|S_t=s] \tag{1} \label{valf}
\end{equation}
$$
One can also defined
$$
\begin{equation}
q_{\pi}(s,a)=\mathbb{E}_{\pi}[G_t|S_t=s, A_t=a] \tag{2} \label{action_valf}
\end{equation}
$$
Then, we can write 
$$
\begin{align}
G_t&=R_{t+1} +\gamma(R_{t+2}+\gamma R_{t+3}+...+\gamma^{T-t-2}R_{T}) \\
&=R_{t+1}+\gamma G_{t+1}
\end{align}
$$

Hence, \eqref{valf} will take take the following form
$$
\begin{align}
v_{\pi}(s) & = \mathbb{E}_{\pi}[G_t|S_t=s] \\ 
&= \mathbb{E}_{\pi}[R_{t+1}+rG_{t+1}|S_t=s]\\
&= \mathbb{E}_{\pi}[R_{t+1}|S_t=s] +\gamma \mathbb{E}_{\pi}[\mathbb{E}_{\pi}[G_{t+1}|S_{t+1}]|S_t=s] \\
&=\mathbb{E}_{\pi}[R_{t+1}+\gamma v_{\pi}(S_{t+1})|S_t=s] \\
&=\sum_{a} \pi(a|s) \sum_{ r,s'} \mathbb{P}(r,s'|a,s) (r + \gamma  v_{\pi}(s') )   \tag{3} \label{analytic_valf}
\end{align}
$$

For the same reason, one can write the `action value` function as follows: 
$$
\begin{align}
q_{\pi}&=\mathbb{E}_{\pi}[G_t|S_t=s,A_t=a] \\ 
&= \sum_{r,s'} \mathbb{P}(s',r|s,a)[r+\gamma v_{\pi}(s')] \tag{4} \label{analytic_action_valf}
\end{align}
$$
Then naturally, one has the following relationship:
$$v_{\pi}(s)=\sum_{a}\pi(a|s) q_{\pi} (s,a)$$
Then, the following relationship is attained
$$
\begin{equation}
q_{\pi}(s,a)=\sum_{ r,s'} \mathbb{P}(r,s'|a,s) (r + \gamma  \sum_{a'}\pi(a'|s') q_{\pi} (s',a') ) \tag{5} \label{q_valf}
\end{equation}
 $$
 
 As a result, the following goal of maximization is in order
$$
\begin{equation}
V_*(s)=\max_{\pi}v_{\pi}(s)
\end{equation}
$$
Also, by the defintion of $q_{\pi}(s,a)$, we have that 
$$
\begin{equation}
V_*(s)=\max_{a}q_{\pi^*}(s,a)
\end{equation}
$$
Hence, compared to \eqref{analytic_valf} we have that the best policy is to put all the weight on the location $\mathbb{P}(a|s)$ where the term on the right handside \eqref{opt_1} is optimized
$$
\begin{align}
v_{*}(s) &= \max_a \sum_{ r,s'} \mathbb{P}(r,s'|a,s) (r + \gamma  v_{\pi}(s') ) \tag{6} \label{opt_1} \\ 
q_*(s,a) &=  \sum_{ r,s'} \mathbb{P}(r,s'|a,s) (r + \gamma  \max_{a'}q_{*}(s',a') )  \tag{7} \label{opt_2}
\end{align}
$$

Basically, the priciple is that the optimal policy "must" take
$$\pi^*(A=a'|s)=1$$
if given $s$, the action $a'$ is the one that maximizes $v(s)$ among all other actions.

## Model Based Algorithms

In this case, we assume that the `transition dynamics is known`. This is however typically a very strong assumption and may not be true under most scenarios.
$$\mathbb{P}(S_t=s', R_t=r| S_{t-1}=s, A_{t-1}=a)$$
Problem like this are typically discrete, 
1. $n[A]$ is the total number of actions
2. $n[S]$ is the total number of states
3. The dictionary $P[s][a]$ assigns probability to such a pair.

#### <ins>Policy Iterations</ins>
Recall the Bellman's equation:
$$
\begin{equation}
v_{\pi}=\sum_{a} \pi(a|s) \sum_{ r,s'} \mathbb{P}(r,s'|a,s) (r + \gamma  v_{\pi}(s') ) \tag{BE} \label{be}
\end{equation}
$$
And one should have total $n[S]$ of such equations. This forms a very large linear system when the cardinality is large. 
Since finding the solution of a large system is difficult in general, we seek to the contraction mapping idea. So the hope is that as $k \rightarrow \infty$, $v_k \rightarrow v_{\pi^*}$.
This approach of finding all the value functions for the states is called `Policy Evalutions`. 
$$
\begin{equation}
v_{k+1}(s) \leftarrow \sum_{a} \pi(a|s) \sum_{ r,s'} \mathbb{P}(r,s'|a,s) (r + \gamma  v_{k}(s') ) \tag{BE update} \label{be_u}
\end{equation}
$$

#### <ins>Policy improvement and Iterations</ins>
The state-action function can be computed as follows
$$
\begin{equation}
q(s,a) =  \sum_{ r,s'} \mathbb{P}(r,s'|a,s) (r + \gamma  v_{\pi}(s') )
\end{equation}
$$
Then we have 
$$v_{\pi^*}(s)=\max_{a} q_{\pi^*}(s,a)$$
which in turn leads to the following results:
$$\pi'(a|s)=argmax_{a} q(s,a)$$
There is a `policy improvment theorem` that says: the values for all states under the new Policy $\pi'$ is equal to or greateer than the state value function under the orignal policy unless it is the optimal one. 

Hence, one can update the policy in the following fashion: 
$$\pi_0 \rightarrow v_{\pi_0} \rightarrow \pi_1 (\text{from} \ q(s,a)) \rightarrow v_{\pi_1} \rightarrow ... \rightarrow \pi_{*} \rightarrow v_{*} $$ 
And one should have consistently $v_{\pi_{k+1}} \geq v_{\pi_k}$

### `Algorithm: Policy Iteration`:

Set v(s)=0, s $\in$ S

Loop until $\Delta \leq \delta$: 
$$
\begin{align}
v'_{\pi}(s) &\leftarrow \sum_{a} \pi(a|s) \sum_{ r,s'} \mathbb{P}(r,s'|a,s) (r + \gamma  v_{\pi}(s') ),   \ \forall s \in S \\
\Delta &= \max(\Delta, |v'(s)-v(s)|)
\end{align}
$$

$v(s) \leftarrow v(s')$ , $\forall s \in S$

Policy changed $\leftarrow$ False 

Loop $\forall s \in S$:

$$
\begin{align}
\text{old-action} &\leftarrow \pi(s)\\ 
\pi(s) &\leftarrow argmax_{a} \sum_{s', r} \mathbb{P}(r,s'|a,s) (r + \gamma  v_{\pi}(s') )\\
\text{If} \pi(s) &\neq \text{old-action}, \ \ \text{Policy-changed = True }
\end{align}
$$

One obvious downside of the algorithm above is that it requires two separate steps for an update of the policy. The value evalution step can take long by using the contraction idea.


`The following approach is called "value iteration"`

### `Algorithm: Value Iteration`:

Set v(s)=0, s $\in$ S

Loop until $\Delta \leq \delta$: 
$$
\begin{align}
v'_{\pi}(s) &\leftarrow \max_a  \sum_{ r,s'} \mathbb{P}(r,s'|a,s) (r + \gamma  v_{\pi}(s') ),   \ \forall s \in S \\
\Delta &= \max(\Delta, |v'(s)-v(s)|)
\end{align}
$$

Find the optimal policy
$$\pi(s) \leftarrow argmax_{a} \sum_{s', r} \mathbb{P}(r,s'|a,s) (r + \gamma  v(s') )$$
